In [ ]:
import pandas as pd
import numpy as np
import json

from utils import load

idx = pd.IndexSlice

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from keras.layers import Input, Dense
from keras.models import Model

# from autoencoder_base import AutoencoderBase

In [ ]:
from keras.callbacks import EarlyStopping, TensorBoard

class AutoencoderBase(object):

    def train(self, x_train, x_test, epochs, batch_size, log_dir='/tmp/autoencoder', stop_early=True):
        callbacks = []
        if backend._BACKEND == 'tensorflow':
            callbacks.append(TensorBoard(log_dir=log_dir))

#     if stop_early:
#         callbacks.append(EarlyStopping(monitor='val_loss', patience=2, verbose=1, mode='auto'))

        self.autoencoder.fit(x_train, x_train,
                nb_epoch=epochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(x_test, x_test),
                callbacks=callbacks)

    def encode(self, x):
        return self.encoder.predict(x)

    def decode(self, x):
        return self.decoder.predict(x)

    def summary(self):
        self.autoencoder.summary()

In [ ]:
class Autoencoder(AutoencoderBase):

    def __init__(self, dim_in, encoding_dim):
        input_img = Input(shape=(dim_in,), name='EncoderIn')

        encoded = Dense(encoding_dim, activation='relu', name='Encoder')(input_img)

        decoded = Dense(dim_in, activation='sigmoid', name='Decoder')(encoded)

        self.autoencoder = Model(input=input_img, output=decoded)

        self.encoder = Model(input=input_img, output=encoded)

        encoded_input = Input(shape=(encoding_dim,), name='DecoderIn')
        decoder_layer = self.autoencoder.layers[-1]
        self.decoder = Model(input=encoded_input, output=decoder_layer(encoded_input))

        self.autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

In [ ]:
 class DeepAutoencoder(AutoencoderBase):

    def __init__(self, dims):
        dim_in = dims[0]
        dim_out = dims[-1]
        dims_encoder = dims[1:]
        dims_decoding = dims[:-1]
        dims_decoding.reverse()

        input_img = Input(shape=(dim_in,), name='EncoderIn')
        decoder_input = Input(shape=(dim_out,), name='DecoderIn')

        encoded = input_img

        # Construct encoder layers
        for i, dim in enumerate(dims_encoder):
            name = 'Encoder{0}'.format(i)
            encoded = Dense(dim, activation='relu', name=name)(encoded)

        # Construct decoder layers
        # The decoded is connected to the encoders, whereas the decoder is not
        decoded = encoded
        decoder = decoder_input
        for i, dim in enumerate(dims_decoding):
            name = 'Decoder{0}'.format(i)

            activation = 'relu'
            if i == len(dims_decoding) - 1:
                activation = 'sigmoid'

            layer = Dense(dim, activation=activation, name=name)

            decoded = layer(decoded)
            decoder = layer(decoder)

        self.autoencoder = Model(input=input_img, output=decoded)
        self.encoder = Model(input=input_img, output=encoded)
        self.decoder = Model(input=decoder_input, output=decoder)

        self.autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy')

# Load dataset 

In [ ]:
import librosa as lbra
import librosa.display
import IPython.display as ipd

from tools import id_path

In [ ]:
songs = pd.read_pickle('electro_index.pkl')

In [ ]:
electronly = songs[songs['genres'].map(lambda x: x == [15])].copy()

In [ ]:
results = np.empty((1, 661560))
for i, song in enumerate(electronly.index.get_level_values(0)):
    wave, sr = lbra.load('songs/electro/fma_large/' + id_path(song))
#     results = np.append(results, [[wave.tolist(), [sr]]], axis=0)
    results = np.append(results, [wave], axis=0)
    print(f'\r {round(i/electronly.shape[0]*100, 2)} %', end='')

In [ ]:
np.array([wave]).shape

In [ ]:
np.append(np.array([]), [wave])

In [ ]:
results[(0, 1)]

In [ ]:
len(results[(0, 0)])

In [ ]:
len(results[-3])

In [ ]:
lbra.display.waveplot(song)

In [ ]:
ipd.Audio(song, rate=sr)

In [ ]:
autoencoder = Autoencoder(dim_in)